# Classificazione pagine wikipedia

Codice per classificare le pagine di wikipedia nella versione inglese tramite l'utilizzo dei nodi wikidata associati alle pagine (laddove presenti ed utilizzabili) e il classificatore di pagine [ORES](https://www.mediawiki.org/wiki/ORES).
Le pagine da classificare saranno quelle presenti nelle pagine più visitate nei mesi tra il 2016-2020 e saranno caricate su MongoDB in una collection a parte.

Per ogni pagina sarà generato un documento strutturrato come il seguente (esempio David Bowie):
``` python
{
    "_id":"8786", # page_id
    "article":"David_Bowie", # page title
    "category":"arts, culture and entertainment", # category with taxonomy ad hoc
    "description":"British singer, musician, and actor (1947-2016)", # description 
    "node_id":"Q5", # node id type
    "ores_cat":"Culture.Media.Music", # cateogry predicted by ores classifier
    "page_type":"biography", # type of the page (other or biography)
    "rev_id": 1015654584, # id last revison
    "wikidata_id":"Q5383" # id page associated wikidata node
}
```

## Carico funzioni e librerie necessarie
Alcune funzioni sono presenti nella repository github del progetto al seguente [link](https://github.com/riccardorubini98/wikitrend).

In [ ]:
pip install dnspython

In [ ]:
import requests
import time as time
from tqdm.notebook import tqdm
from requests.exceptions import ConnectionError
# Import github repository
!git clone https://github.com/riccardorubini98/wikitrend.git
import sys
sys.path.append("/content/wikitrend/")
import classifier_function as ct

# Collegamento a MongoDB
Mi collego a MongoDB Atlas e scarico la collezione contente le pagine più visitate mese per mese.

In [ ]:
# Connect mongodb atlas
from pymongo import MongoClient
access_token = ""
client = MongoClient(access_token)
db = client.get_database('wikitrend')
records = db.toppage.find() # top page for month
pages = db.page_topic # new collection for single pages

# Creazione loop per classificare le pagine
Creo un ciclo for per ogni documento e per ogni pagina presente nella collezione contenente le pagine più viste per mese. Se la pagina presa in esame non è già stata classificata allora procedo alla categorizzazione, in caso contrario passo alla pagina successiva.

L'algoritmo per la classificazione funziona in questo modo:
1. Assegno gli ids necessari alla pagina per essere classificata. Fra questi può essere presente l'id del nodo wikidata associato.
2. Se l'id wikidata è presente, cerco la tipologia del nodo associato alla pagina e in base a questa definisco la tipologia della pagina (biografica o altro) e in alcuni casi già la categoria ORES associata (ad esempio se il nodo è relativo ad un Film la categoria ORES sarà senz'altro *culture.media.films*)
3. Per le pagina che con il punto precedente non sono state classificate utilizzo il classificatore ORES con il metodo *draft topic*. Qual ora questo non producesse risultati utili eseguo un nuovo tentativo con il metodo *article topic*.
4. Converto la categoria ORES nella tassonomia utilizzata nel progetto. Le nuove cateogire sono: art, culture and entertainment; history and geography, internet and game, sport, STEM and other.

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

for record in records[29:]:
    start = time.perf_counter()
    print("Analyze record: ", record["_id"],"\n")
    top_month = record["top_month"]
    top_day = record["top_day"]
    
    # select unique page from top_month and top_day list
    id_list = []
    page_list = []
    for articolo in top_month:
      id_list.append(articolo["page_id"])
      page_list.append(articolo)
    for articolo in top_day:
      page_id = articolo["page_id"]
      if page_id not in id_list:
        page_list.append(articolo)

    # find new pages to categorize and upload
    new_pages = []
    for articolo in tqdm(page_list,desc='Find new pages \n'):
        page_id = articolo["page_id"]
        # pages without page_id
        if page_id == -1 or page_id == None:
            pass
        else:
            # search in mongodb collection
            find_page = pages.find_one({"_id": page_id})
            # if there isn't any page with the same _id
            if find_page == None:
                new_pages.append(articolo)
            else:
                pass
    
    print("New page: ", len(new_pages), "\n")  
    
    # categorizing new pages
    # adding ids
    s = requests.session()
    s.headers.update(headers)
    for articolo in tqdm(new_pages,desc='Adding ids \n'):
        url = ct.api_id(articolo["article"])
        # make api call
        i=0
        not_found=True
        while i <30 and not_found:
            try:
                r = s.get(url=url,timeout=20)
                not_found=False
            except ConnectionError:
                time.sleep(1)
                i += 1
        r= r.json()
        # Adding id to page document
        articolo["page_id"] = ct.get_pageid(r)
        articolo["rev_id"] = ct.get_revid(r)
        articolo["wikidata_id"] = ct.get_wikidataid(r)
    
    # get node_type id
    s = requests.session()
    s.headers.update(headers)
    for articolo in tqdm(new_pages,desc="Adding node type \n"):
        if "wikidata_id" in articolo.keys() and articolo["wikidata_id"] != None:
            url = ct.api_nodeid(articolo["wikidata_id"])
            # make api call
            i=0
            not_found=True
            while i <30 and not_found:
                try:
                    r = s.get(url=url,timeout=20)
                    not_found=False
                except ConnectionError:
                    time.sleep(1)
                    i += 1
            r= r.json()
            node_id = ct.get_nodeid(r)
            articolo["node_id"] = node_id
            if node_id != None:
                nodetype = ct.get_nodetype(node_id)
                if nodetype == None:
                    pass
                else:
                    if len(nodetype) == 1:
                        articolo["page_type"] = nodetype[0]
                    elif len(nodetype) == 2:
                        articolo["page_type"] = nodetype[0]
                        articolo["ores_cat"] = nodetype[1]
            else:
                pass

    # get_category
    s = requests.session()
    s.headers.update(headers)
    for articolo in tqdm(new_pages, desc="Getting category \n"):
        if "ores_cat" in articolo.keys():
            articolo["category"] = ct.ores_converter(pred_cat)
        else:
            url = ct.api_draftcat(articolo["rev_id"])
            i=0
            not_found=True
            while i <30 and not_found:
                try:
                    r = s.get(url=url)
                    not_found=False
                except ConnectionError:
                    time.sleep(1)
                    i += 1
            r= r.json()
            cat = ct.get_draftcat(r)
            # If there is a cateogry of biographical type then the page type is "biography"
            if 'Culture.Biography.Biography*' in cat or 'Culture.Biography.Women' in cat:
                articolo["page_type"] = "biography"            
            draft_cat = ct.clean_cat(cat)
            # If with draft method I don't get a useful category I try with article method
            if draft_cat[:3] == "Geo" or draft_cat == "other.category":
                url = ct.api_articlecat(articolo["rev_id"])
                i=0
                not_found=True
                while i <30 and not_found:
                    try:
                        r = s.get(url=url)
                        not_found=False
                    except ConnectionError:
                        time.sleep(1)
                        i += 1
                r= r.json()
                cat2 = ct.get_articlecat(r)      
                article_cat = ct.clean_cat(cat2)
                # select best category from draft and article method
                pred_cat = ct.clean_cat([draft_cat,article_cat])
            else:
                pred_cat = draft_cat
            # If a page hasn't page_type then it's "other"
            if "page_type" not in articolo.keys():
                articolo["page_type"] = "other"
            # If a page is biography and the predicted category is of geographical type then that page isn't classified
            if articolo["page_type"] == "biography" and pred_cat[:3] == "Geo":
                    pred_cat = "other.category"
            # Save category
            articolo["ores_cat"] = pred_cat
            # Convert category
            articolo["category"] = ct.ores_converter(pred_cat)

    # Upload new page on MongoDB
    for page in tqdm(new_pages,desc= "Uploading pages \n"):
        page["_id"] = page["page_id"]
        del page["page_id"]
        del page["views"]
        if "day_of_month" in page.keys():
            del page["day_of_month"]
        try:
            pages.insert_one(page)
        except:
            print(page)